<a href="https://colab.research.google.com/github/koheikobayashi/bertsum/blob/main/BERTSum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy
!pip install transformers
!pip install neuralcoref
!pip install bert-extractive-summarizer
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.2 MB/s 
     |████████████████████████████████| 6.6 MB 33.7 MB/s 
     |████████████████████████████████| 120 kB 57.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 286 kB 7.9 MB/s 
     |████████████████████████████████| 132 kB 68.4 MB/s 
     |████████████████████████████████| 79 kB 9.2 MB/s 
     |████████████████████████████████| 9.1 MB 61.5 MB/s 
     |████████████████████████████████| 127 kB 55.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/

In [2]:
from summarizer import Summarizer

model=Summarizer()
text='It is not the strongest of the species that survives, nor the most intelligent that survives. It is the one that is most adaptable to change.'
generate_text=model(text, min_length=60, max_length=750, use_first='First')
print(generate_text)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


It is not the strongest of the species that survives, nor the most intelligent that survives.


In [ ]:
!pip install -qU torch==1.7.1 torchtext==0.8.0 torchvision==0.8.2
!pip install -q transformers==4.2.2 sentencepiece

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import AdamW,get_linear_schedule_with_warmup

# 学習済みモデルをHugging Face model hubからダウンロードする
model_dir_name = "sonoisa/t5-qiita-title-generation"

# トークナイザー（SentencePiece）
tokenizer = T5Tokenizer.from_pretrained(model_dir_name, is_fast=True)

# 学習済みモデル
trained_model = T5ForConditionalGeneration.from_pretrained(model_dir_name)

# GPUの利用有無
USE_GPU = torch.cuda.is_available()
if USE_GPU:
  trained_model.cuda()

In [ ]:
# https://github.com/neologd/mecab-ipadic-neologd/wiki/Regexp.ja から引用・一部改変
from __future__ import unicode_literals
import re
import unicodedata

def unicode_normalize(cls, s):
    pt = re.compile('([{}]+)'.format(cls))

    def norm(c):
        return unicodedata.normalize('NFKC', c) if pt.match(c) else c

    s = ''.join(norm(x) for x in re.split(pt, s))
    s = re.sub('－', '-', s)
    return s

def remove_extra_spaces(s):
    s = re.sub('[ 　]+', ' ', s)
    blocks = ''.join(('\u4E00-\u9FFF',  # CJK UNIFIED IDEOGRAPHS
                      '\u3040-\u309F',  # HIRAGANA
                      '\u30A0-\u30FF',  # KATAKANA
                      '\u3000-\u303F',  # CJK SYMBOLS AND PUNCTUATION
                      '\uFF00-\uFFEF'   # HALFWIDTH AND FULLWIDTH FORMS
                      ))
    basic_latin = '\u0000-\u007F'

    def remove_space_between(cls1, cls2, s):
        p = re.compile('([{}]) ([{}])'.format(cls1, cls2))
        while p.search(s):
            s = p.sub(r'\1\2', s)
        return s

    s = remove_space_between(blocks, blocks, s)
    s = remove_space_between(blocks, basic_latin, s)
    s = remove_space_between(basic_latin, blocks, s)
    return s

def normalize_neologd(s):
    s = s.strip()
    s = unicode_normalize('０-９Ａ-Ｚａ-ｚ｡-ﾟ', s)

    def maketrans(f, t):
        return {ord(x): ord(y) for x, y in zip(f, t)}

    s = re.sub('[˗֊‐‑‒–⁃⁻₋−]+', '-', s)  # normalize hyphens
    s = re.sub('[﹣－ｰ—―─━ー]+', 'ー', s)  # normalize choonpus
    s = re.sub('[~∼∾〜〰～]+', '〜', s)  # normalize tildes (modified by Isao Sonobe)
    s = s.translate(
        maketrans('!"#$%&\'()*+,-./:;<=>?@[¥]^_`{|}~｡､･｢｣',
              '！”＃＄％＆’（）＊＋，－．／：；＜＝＞？＠［￥］＾＿｀｛｜｝〜。、・「」'))

    s = remove_extra_spaces(s)
    s = unicode_normalize('！”＃＄％＆’（）＊＋，－．／：；＜＞？＠［￥］＾＿｀｛｜｝〜', s)  # keep ＝,・,「,」
    s = re.sub('[’]', '\'', s)
    s = re.sub('[”]', '"', s)
    return s

In [ ]:
import re

CODE_PATTERN = re.compile(r"```.*?```", re.MULTILINE | re.DOTALL)
LINK_PATTERN = re.compile(r"!?\[([^\]\)]+)\]\([^\)]+\)")
IMG_PATTERN = re.compile(r"<img[^>]*>")
URL_PATTERN = re.compile(r"(http|ftp)s?://[^\s]+")
NEWLINES_PATTERN = re.compile(r"(\s*\n\s*)+")

def clean_markdown(markdown_text):
    markdown_text = CODE_PATTERN.sub(r"", markdown_text)
    markdown_text = LINK_PATTERN.sub(r"\1", markdown_text)
    markdown_text = IMG_PATTERN.sub(r"", markdown_text)
    markdown_text = URL_PATTERN.sub(r"", markdown_text)
    markdown_text = NEWLINES_PATTERN.sub(r"\n", markdown_text)
    markdown_text = markdown_text.replace("`", "")
    return markdown_text

def normalize_text(markdown_text):
    markdown_text = clean_markdown(markdown_text)
    markdown_text = markdown_text.replace("\t", " ")
    markdown_text = normalize_neologd(markdown_text).lower()
    markdown_text = markdown_text.replace("\n", " ")
    return markdown_text

def preprocess_questionnaire_body(markdown_text):
    return "body: " + normalize_text(markdown_text)[:4000]

In [ ]:
def postprocess_abstract(abstract):
  return re.sub(r"abstract: ", "", abstract)

In [ ]:
questionnaire_body = """
私は、より多様な情報に触れたいから、また国際会議などの場で外国人と議論を交わしたいからという理由で英語を学んでいる。
"""

In [ ]:
preprocess_questionnaire_body(questionnaire_body)

In [ ]:
MAX_SOURCE_LENGTH = 512  # 入力文の最大トークン数
MAX_TARGET_LENGTH = 64   # 生成文の最大トークン数

# Pytorchのモデルを訓練モードから評価モードに変更
trained_model.eval()

# 入力文の前処理とトークナイズを行う
inputs = [preprocess_questionnaire_body(questionnaire_body)]
batch = tokenizer.batch_encode_plus(
    inputs, max_length=MAX_SOURCE_LENGTH, truncation=True, 
    padding="longest", return_tensors="pt")

input_ids = batch['input_ids']
input_mask = batch['attention_mask']
if USE_GPU:
  input_ids = input_ids.cuda()
  input_mask = input_mask.cuda()

# 出力文の生成処理を行う
outputs = trained_model.generate(
    input_ids=input_ids, attention_mask=input_mask, 
    max_length=MAX_TARGET_LENGTH,
    return_dict_in_generate=True, output_scores=True,
    temperature=1.0,            # 生成にランダム性を入れる温度パラメータ
    num_beams=10,               # ビームサーチの探索幅
    diversity_penalty=1.0,      # 生成結果の多様性を生み出すためのペナルティ
    num_beam_groups=10,         # ビームサーチのグループ数
    num_return_sequences=5,    # 生成する文の数
    repetition_penalty=1.5,     # 同じ文の繰り返し（モード崩壊）へのペナルティ
)

# 生成されたトークンIDを文字列に変換する
generated_sentences = [tokenizer.decode(ids, skip_special_tokens=True, 
                                     clean_up_tokenization_spaces=False) 
                    for ids in outputs.sequences]

# 生成された文案を表示する
for i, sentence in enumerate(generated_sentences):
  print(f"{i+1:2}. {postprocess_sentence(sentence)}")